In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import Audio

import torch
import torch.nn as nn
import torch.nn.functional as F

from einops import rearrange, reduce, repeat

import numpy as np

import librosa

In [ ]:
from models.modules.loss import CrepeFeaturesAndCents
from models.modules.utils import pad_audio, get_frames
from models.modules.harmonic_oscillator import OscillatorBank

In [ ]:
audio, sr = librosa.load('/home/kureta/Music/cello/Cello Samples/ArpAm-00000-.wav', sr=16000, mono=True)

In [ ]:
win_length = 1024
hop_length = 512

In [ ]:
Audio(audio, rate=sr)

In [ ]:
x = torch.from_numpy(audio)
x = x.unsqueeze(0)

In [ ]:
padded_audio = pad_audio(x, win_length, hop_length, strict=False)

In [ ]:
frames = get_frames(padded_audio, win_length, hop_length)

In [ ]:
b, n, x = frames.shape
frames = rearrange(frames, 'b n x -> (b n) x')

In [ ]:
encoder = CrepeFeaturesAndCents()

In [ ]:
frames = frames.cuda()
encoder = encoder.cuda()

In [ ]:
cents, features = encoder(frames)

In [ ]:
cents.shape, features.shape

In [ ]:
cents = rearrange(cents, '(b n) x -> b n x', b=b, n=n)
features = rearrange(features, '(b n) ch w h -> b n (ch w h)', b=b, n=n)

In [ ]:
cents.shape, features.shape

In [ ]:
osc = OscillatorBank(hop_length=hop_length).cuda()

In [ ]:
with torch.no_grad():
    y = osc(
        cents,
        torch.ones_like(cents),
        torch.ones(*cents.shape, osc.n_harmonics, device='cuda')
    )

In [ ]:
y.shape
yy = y[0, :, 0].cpu().numpy()

In [ ]:
Audio(yy, rate=sr)